<a href="https://colab.research.google.com/github/israinamdar493-ops/retinalprojectsizz4/blob/main/Retinalai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
base_path = "/content/drive/MyDrive/retinalnewproject"
print(os.listdir(base_path))


['datasets', 'outputs', 'reports', 'preprocessing', 'segmentation', 'federated', 'ensemble', 'results']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import densenet121, DenseNet121_Weights
from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Training on:", device)

# Load pretrained DenseNet121
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
model.classifier = nn.Linear(model.classifier.in_features, 3)  # 3 classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def evaluate(model, loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = outputs.max(1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total

num_epochs = 10
best_acc = 0
save_path = "/content/drive/MyDrive/retinalnewproject/results/odir3_densenet121.pth"

for epoch in range(num_epochs):
    model.train()
    running_loss = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix({"loss": running_loss / len(train_loader)})

    val_acc = evaluate(model, val_loader)
    print(f"Validation Accuracy: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print("✔ Best model saved!")

print("\nTraining complete!")
print("Best validation accuracy:", best_acc)
print("Model saved at:", save_path)

In [ ]:
!pip install diffusers accelerate transformers --quiet

import torch
from torch import nn
from torchvision import transforms
from diffusers import DDPMScheduler, UNet2DModel
from torch.utils.data import DataLoader
import os
from PIL import Image


In [ ]:
# === Run this single cell in Colab ===
# installs
!pip install -q diffusers accelerate transformers

# imports
import os, sys, random, math
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from diffusers import DDPMScheduler, UNet2DModel

# === SETTINGS ===
image_size = 256
batch_size = 8            # keep small for Colab GPU; increase if you have >12GB GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# paths (uses your existing split_data folders)
BASE_ODIR = "/content/drive/MyDrive/retinalnewproject/datasets/ODIR5K/split_data"
TRAIN_DIR = os.path.join(BASE_ODIR, "train")
VAL_DIR   = os.path.join(BASE_ODIR, "val")

RESULTS = "/content/drive/MyDrive/retinalnewproject/generated/odir_ddpm_256"
os.makedirs(RESULTS, exist_ok=True)

print("image_size:", image_size, "batch_size:", batch_size, "device:", device)
print("TRAIN_DIR exists:", os.path.exists(TRAIN_DIR))
print("VAL_DIR exists:", os.path.exists(VAL_DIR))
print("Results folder:", RESULTS)

# === Dataset that loads only the three folders (diabetes, glaucoma, ageDegeneration) ===
CLASS_MAP = {"diabetes":0, "glaucoma":1, "ageDegeneration":2}
class DDPMDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        for name in CLASS_MAP.keys():
            folder = os.path.join(root_dir, name)
            if not os.path.exists(folder):
                continue
            for f in os.listdir(folder):
                if f.lower().endswith((".png",".jpg",".jpeg")):
                    self.samples.append(os.path.join(folder,f))
        print("Found", len(self.samples), "images in", root_dir)
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        p = self.samples[idx]
        img = Image.open(p).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img

# transforms for diffusion: scale to [-1,1]
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.CenterCrop((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])   # maps [0,1] -> [-1,1]
])

train_ds = DDPMDataset(TRAIN_DIR, transform=transform)
val_ds   = DDPMDataset(VAL_DIR, transform=transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# quick sanity check: show one batch shape and a grid saved to DRIVE
batch = next(iter(train_loader))
print("Batch tensor shape:", batch.shape)   # expected [B,3,H,W]

# save a small grid to RESULTS for quick visual check
grid = utils.make_grid((batch[:8]*0.5+0.5).clamp(0,1), nrow=4)  # undo [-1,1] to [0,1]
utils.save_image(grid, os.path.join(RESULTS, "sample_batch_grid.png"))
print("Saved sample grid to:", os.path.join(RESULTS, "sample_batch_grid.png"))

# check diffusers UNet available (not building/training yet)
unet = UNet2DModel(sample_size=image_size, in_channels=3, out_channels=3, layers_per_block=2)
print("UNet model created (parameters):", sum(p.numel() for p in unet.parameters()) )
print("Ready for next step: build scheduler + small training loop.")


In [ ]:
# Diffusion Scheduler
import torch
import torch.nn.functional as F
from tqdm import tqdm

timesteps = 1000
beta_start = 0.0001
beta_end = 0.02

betas = torch.linspace(beta_start, beta_end, timesteps).to(device)
alphas = 1 - betas
alpha_hat = torch.cumprod(alphas, dim=0)  # cumulative product

print("Scheduler ready - timesteps:", timesteps)


In [ ]:
# === Quick DDPM training + sampling (run 200 steps) ===
import torch, os, math, random
from tqdm import trange
from torchvision import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# reuse variables created earlier in your session:
# - unet (UNet2DModel instance)
# - train_loader
# - betas, alphas, alpha_hat (scheduler tensors)
# - RESULTS path

# Safety checks / defaults if not present
try:
    unet
except NameError:
    raise RuntimeError("UNet object not found in the session. Run the UNet creation cell first.")

if 'RESULTS' not in globals():
    RESULTS = "/content/drive/MyDrive/retinalnewproject/generated/odir_ddpm_256"
os.makedirs(RESULTS, exist_ok=True)

# training hyperparams (fast demo)
num_train_steps = 200       # total gradient steps (small demo)
save_every = 50
sample_every = 50
lr = 2e-4

optimizer = torch.optim.Adam(unet.parameters(), lr=lr)

T = betas.shape[0]
print(f"Using T={T} diffusion steps. Running {num_train_steps} training steps.")

global_step = 0
unet.to(device)
unet.train()

# helper: q_sample (adds noise according to alpha_hat at selected timesteps)
def q_sample(x_start, t, noise):
    # x_start: [B,3,H,W], t: [B] long tensor with values in [0..T-1], noise same shape
    a_hat = alpha_hat[t].view(-1,1,1,1).to(device)   # (B,1,1,1)
    sqrt_a_hat = torch.sqrt(a_hat)
    sqrt_one_minus = torch.sqrt(1. - a_hat)
    return sqrt_a_hat * x_start + sqrt_one_minus * noise

# training loop (iterate data loader repeatedly until num_train_steps)
loader_iter = iter(train_loader)
pbar = trange(num_train_steps, desc="DDPM steps")
for step in pbar:
    try:
        batch = next(loader_iter)
    except StopIteration:
        loader_iter = iter(train_loader)
        batch = next(loader_iter)
    x0 = batch.to(device)           # images already normalized to [-1,1]
    bsz = x0.shape[0]

    # sample random timesteps for each sample
    t = torch.randint(0, T, (bsz,), device=device).long()

    # sample noise and create x_t
    noise = torch.randn_like(x0).to(device)
    x_t = q_sample(x0, t, noise)

    # forward through unet -> predict noise
    out = unet(x_t).sample if hasattr(unet(x_t), "sample") else unet(x_t)
    pred_noise = out

    loss = torch.nn.functional.mse_loss(pred_noise, noise)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    global_step += 1
    pbar.set_postfix({"loss": f"{loss.item():.6f}", "step": global_step})

    # save checkpoint
    if global_step % save_every == 0 or global_step == num_train_steps:
        ckpt_path = os.path.join(RESULTS, f"ddpm_unet_step{global_step}.pt")
        torch.save({'step': global_step, 'model_state': unet.state_dict(), 'opt_state': optimizer.state_dict()}, ckpt_path)
        print(f"\nSaved checkpoint: {ckpt_path}")

    # sampling (simple ancestral sampling)
    if global_step % sample_every == 0 or global_step == num_train_steps:
        unet.eval()
        n_samples = 8
        x = torch.randn(n_samples, 3, image_size, image_size, device=device)  # start from noise
        with torch.no_grad():
            for step_t in reversed(range(T)):
                t_batch = torch.full((n_samples,), step_t, device=device, dtype=torch.long)
                out = unet(x).sample if hasattr(unet(x), "sample") else unet(x)
                pred_noise = out

                alpha_t = alphas[step_t].to(device)
                alpha_cum_t = alpha_hat[step_t].to(device)
                beta_t = betas[step_t].to(device)

                if step_t > 0:
                    noise_term = torch.randn_like(x)
                else:
                    noise_term = torch.zeros_like(x)

                # simplified posterior update
                x = (1.0 / torch.sqrt(alpha_t)) * (x - ((1 - alpha_t) / torch.sqrt(1 - alpha_cum_t)) * pred_noise) + torch.sqrt(beta_t) * noise_term

            samples = (x.clamp(-1,1) + 1) / 2.0   # to [0,1]
            grid = utils.make_grid(samples.cpu(), nrow=4)
            sample_path = os.path.join(RESULTS, f"sample_step{global_step}.png")
            utils.save_image(grid, sample_path)
            print(f"Saved sample grid: {sample_path}")
        unet.train()

print("\nDDPM quick run finished.")
print("All samples and checkpoints saved to:", RESULTS)
